# Steady State Solve

Example is based on Julian's Laplace example.

In [1]:
import petsc4py.PETSc as PETSc
import underworld3 as uw

In [2]:
options = PETSc.Options()

In [3]:
options.setValue("dm_plex_separate_marker", None)
options.setValue("temperature_petscspace_degree", 1)
options.setValue("sol_vec_view", None)
options.setValue("dm_view", None)
options.setValue("options_view",None)

In [4]:
user = {"y0": -0.6,
        "y1": 1.3,
        "k": 0.5,
        "h": 10,
        "T0": 4.0,
        "T1": 8.0,
        "simplex": False}

In [5]:
snes = PETSc.SNES().create(comm=PETSc.COMM_WORLD)

## Create Mesh

In [6]:
mesh = uw.FeMesh(elementRes=[10,10], minCoords=(-1.0, user["y0"]), maxCoords=(1.0, user["y1"]), simplex=user["simplex"])

In [7]:
mesh.plex.setFromOptions()

In [8]:
mesh.plex.view()

In [9]:
part = mesh.plex.getPartitioner()
part.setFromOptions()
mesh.plex.distribute()

In [10]:
mesh.plex.localizeCoordinates()
mesh.plex.setFromOptions()

## Setup Discretisation

In [11]:
# Get Communicator from DM
comm = mesh.plex.comm
# Create Default FE
fe_temp = PETSc.FE().createDefault(dim=mesh.dim,
                                   nc=1,
                                   isSimplex=mesh.isSimplex,
                                   prefix="temperature_",
                                   qorder=1, comm=comm)
mesh.plex.setField(0, fe_temp)
mesh.plex.createDS()
prob = mesh.plex.getDS()

### Setup Problems

In [12]:
from underworld3.poisson import PoissonSetup

In [13]:
PoissonSetup(mesh.plex, prob, user)

In [14]:
fe_temp.destroy()

In [15]:
snes.setDM(mesh.plex)

In [16]:
mesh.plex.createClosureIndex(None)

In [17]:
mesh.plex.setSNESLocalFEM()

In [18]:
u = mesh.plex.createGlobalVector()

In [19]:
snes.setFromOptions()

In [20]:
import numpy as np
u[:] = np.ones_like(u.array) * 0.0

In [21]:
u.view()

In [22]:
lu = mesh.plex.getLocalVec()
mesh.plex.insertBoundaryValues(True, lu, 0)
mesh.plex.globalToLocal(u, lu)
lu.view()

In [23]:
u.setName("Initial Solution")
u.setName("Solution")

In [24]:
snes.solve(None, u)

In [25]:
u.view()

In [26]:
viewer = PETSc.Viewer().createHDF5("temperature.h5", "w")
viewer(u)
viewer(mesh.plex)
del viewer
from underworld3.petsc_gen_xdmf import generateXdmf
generateXdmf("temperature.h5")